In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [6]:
import pandas as pd

df = pd.read_csv(
    r"D:\Sudarshan Kasar\Capstone\Twitter Sentiment Analysis\Sentiment_cleaned.csv",
    encoding="latin1",
    header=None
)

df.columns = ['target', 'id', 'date', 'flag', 'user', 'text']


In [4]:
df=df.drop(['id','date','flag','user'],axis=1)

In [7]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\GAURAV
[nltk_data]     DOND\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to C:\Users\GAURAV
[nltk_data]     DOND\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
import re

stop_word=set(stopwords.words('english'))
def clean_txt(text):
    text=re.sub(r'http\S+',' ',text)
    text=re.sub(r'<.*?>',' ',text)
    text=re.sub(r'[^a-z\s]',' ',text)
    text=text.lower().split()
    text=[lemmatizer.lemmatize(word) for word in text if word not in stop_word]
    return ' '.join(text)


df['clean_text']=df['text'].apply(clean_txt)
corpus=df['text'].tolist()

In [11]:
df.head()

,target,id,date,flag,user,text,clean_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot www bummer ou shoulda got avid arr ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset update acebook texting might cry result ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,enichan dived many time ball anaged save rest ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving mad see


In [12]:
df.drop('text',axis=1)

,target,id,date,flag,user,clean_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,switchfoot www bummer ou shoulda got avid arr ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,upset update acebook texting might cry result ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,enichan dived many time ball anaged save rest ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass behaving mad see
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,ust woke aving school best feeling ever
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,com ery cool hear old alt interview
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,ready akeover sk detail
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,appy th irthday boo alll time upac maru hakur


In [13]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)

In [14]:
tokenized_text=tokenizer.texts_to_sequences(corpus)

In [15]:
max_len=max(len(seq) for seq in tokenized_text)

In [16]:
max_len

118

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x=pad_sequences(tokenized_text,maxlen=max_len,padding='post')

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

sample_df = df.sample(n=50000, random_state=42)   # smaller than full data
vec = TfidfVectorizer(max_features=5000)
X = vec.fit_transform(sample_df['clean_text'].astype(str))

tr, te = train_test_split(range(X.shape[0]), test_size=0.3, stratify=sample_df['target'], random_state=42)
X_train, X_test = X[tr], X[te]
y_train = sample_df['target'].iloc[tr].values
y_test  = sample_df['target'].iloc[te].values

clf = LogisticRegression(max_iter=200)
clf.fit(X_train, y_train)
print("test score:", clf.score(X_test, y_test))


test score: 0.7416


In [26]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,df['target'],test_size=0.30,random_state=42)

In [27]:
max_vocab=len(tokenizer.word_index)+1
max_vocab

690961

In [28]:
max_len1= x_train.shape[1]
max_len1

118

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding,Bidirectional

model=Sequential()
model.add(Embedding(max_vocab,output_dim=128,input_length=max_len1))
model.add(Bidirectional(LSTM(150,return_sequences=False)))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

c:\Users\GAURAV DOND\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [30]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop=EarlyStopping(monitor='val_loss',patience=2,restore_best_weights=True)
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
history=model.fit(x_train,y_train,batch_size=64,epochs=50,shuffle=True,validation_split=0.1,callbacks=[early_stop],verbose=1)

In [35]:
model.save('sentiment_model.h5')

In [36]:
import joblib

joblib.dump(tokenizer,'tokenizer.pkl')



['tokenizer.pkl']